In [2]:
# nnictl create --config config.yaml --port 8080

from src.data import CostomerDataset, CostomerDataModule
from src.utils import convert_category_into_integer
from src.model.mlp import Model
from src.training import CostomerModule

import pandas as pd
import numpy as np
import random
import json
import nni
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch

import lightning as L
from lightning.pytorch.trainer import Trainer
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger

import seaborn as sns

In [27]:
from src.data import CostomerDataset, CostomerDataModule
from src.model.mlp import Model
from src.training import CostomerModule

import pandas as pd
import numpy as np
import random
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler



import torch
from torch.utils.data import DataLoader

def main(configs):
    # 데이터 로드 및 전처리
    costomer = pd.read_csv('./data/train.csv')
    costomer = costomer.dropna()

    # 범주형 변수를 숫자로 변환하는 함수
    costomer, _ = convert_category_into_integer(costomer, (
        'Churn', 'ServiceArea', 'ChildrenInHH', 'HandsetRefurbished', 
        'HandsetWebCapable', 'TruckOwner', 'RVOwner', 'Homeownership', 
        'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings', 
        'NonUSTravel', 'OwnsComputer', 'HasCreditCard', 'NewCellphoneUser', 
        'NotNewCellphoneUser', 'OwnsMotorcycle', 'HandsetPrice', 
        'MadeCallToRetentionTeam', 'CreditRating', 'PrizmCode', 
        'Occupation', 'MaritalStatus'
    ))
    costomer = costomer.astype(np.float32)

    # Train/Validation/Test Split
    train, temp = train_test_split(costomer, test_size=0.4, random_state=seed)
    valid, test = train_test_split(temp, test_size=0.5, random_state=seed)

    # 표준화 작업
    standard_scaler = StandardScaler()

    other_columns = ['MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge', 
                     'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls', 
                     'PercChangeMinutes', 'PercChangeRevenues', 'DroppedCalls', 
                     'BlockedCalls', 'UnansweredCalls', 'CustomerCareCalls', 
                     'ThreewayCalls', 'ReceivedCalls', 'OutboundCalls', 
                     'InboundCalls', 'PeakCallsInOut', 'OffPeakCallsInOut', 
                     'DroppedBlockedCalls', 'CallForwardingCalls', 
                     'CallWaitingCalls', 'MonthsInService', 'UniqueSubs', 
                     'ActiveSubs', 'Handsets', 'HandsetModels', 
                     'CurrentEquipmentDays', 'AgeHH1', 'AgeHH2', 'RetentionCalls', 
                     'RetentionOffersAccepted', 'ReferralsMadeBySubscriber', 
                     'IncomeGroup', 'AdjustmentsToCreditRating']

    train.loc[:, other_columns] = standard_scaler.fit_transform(train.loc[:, other_columns])
    valid.loc[:, other_columns] = standard_scaler.transform(valid.loc[:, other_columns])
    test.loc[:, other_columns] = standard_scaler.transform(test.loc[:, other_columns])

    # Dataset과 DataLoader 설정
    train_dataset = CostomerDataset(train)
    valid_dataset = CostomerDataset(valid)
    test_dataset = CostomerDataset(test)

    test_dataloader = DataLoader(
        test_dataset,
        batch_size=configs.get('batch_size'),
    )

    configs.update({'input_dim': len(costomer.columns)-1})
    
    # 모델 로드
    model = Model(configs)
    # model.load_state_dict(torch.load('./model/mlp.pth'))  # 모델 파라미터 불러오기
    model_state_dict = torch.load("./model/mlp.pth", map_location=device)
    model.load_state_dict(model_state_dict)
    # model.eval()  # 평가 모드로 전환

    # prediction
    preds = []
    gts = []
    for batch in test_dataloader:
        X = batch.get('X')
        y = batch.get('y')
        # X = torch.from_numpy(batch.iloc[idx].drop('Churn').values).float()
        # y = torch.Tensor([batch.iloc[idx].Churn]).float()
        with torch.no_grad():  # 예측 시에는 그래디언트 필요 없음
            pred = model(X)
            print(pred)
            preds.append(pred.argmax(dim=-1))
            gts.append(y)

    # 텐서 리스트를 합침
    preds = torch.cat(preds)
    gts = torch.cat(gts)

    # Confusion Matrix 및 성능 지표 계산
    confusion_matrix_result = confusion_matrix(gts.cpu().numpy(), preds.cpu().numpy())
    TP = confusion_matrix_result[0, 0]
    FN = confusion_matrix_result[0, 1]
    FP = confusion_matrix_result[1, 0]
    TN = confusion_matrix_result[1, 1]

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precision * recall / (precision + recall)
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    print(f'TP: {TP: .2f}, TN: {TN: .2f}, FP: {FP: .2f}, FN: {FN: .2f}')

    print(f'precision: {precision: .2f}, recall: {recall: .2f}, f1: {f1: .2f}, accuracy: {accuracy: .2f}')
    print(classification_report(gts, preds))

if __name__ == '__main__':
    # 사용 가능한 GPU가 있는 경우 'cuda', 그렇지 않으면 'cpu' 사용
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # hyperparameter
    with open('./configs.json', 'r') as file:
        configs = json.load(file)
    configs.update({'device': device})

    # seed 설정
    seed = configs.get('seed')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    # CUDA 설정
    if device == 'cuda':
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = True
    
    main(configs)


C:\Users\김정훈\AppData\Local\Temp\ipykernel_17628\3009880447.py:74: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load("./model/mlp.pth", map_location

TP:  7126.00, TN:  0.00, FP:  2825.00, FN:  0.00
precision:  0.72, recall:  1.00, f1:  0.83, accuracy:  0.72
              precision    recall  f1-score   support

         0.0       0.72      1.00      0.83      7126
         1.0       0.00      0.00      0.00      2825

    accuracy                           0.72      9951
   macro avg       0.36      0.50      0.42      9951
weighted avg       0.51      0.72      0.60      9951



c:\Users\김정훈\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\김정훈\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\김정훈\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [22]:
costomer = pd.read_csv('./data/train.csv')
costomer = costomer.dropna()
costomer.Churn = np.where(costomer.Churn == "Yes", 1, 0)
costomer.Churn

0        1
1        1
2        0
3        0
4        1
        ..
51035    0
51037    0
51040    0
51041    1
51043    0
Name: Churn, Length: 49752, dtype: int32